In [ ]:
ua = UserAgent()
res = []
bad_requests = []

def execute_query(qquery, entity_id):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def entities_sparqul(entity_id, f = 0):
        res = []
        if f > 5:
            global bad_requests
            bad_requests.append(entity_id)
            print('Bad request ', entity_id)
            return None
        try:
            entities = {}
            raw_results= execute_query(bookquery.subclass_entity_count, entity_id)
            entities['class'] = i
            entities['count_P31'] = raw_results[0]['entitycount']['value']
            raw_results= execute_query(bookquery.subclass_entity_count2, entity_id)
            entities['count_P31/P279*'] = raw_results[0]['entitycount']['value']
            res.append(entities)
            return pd.DataFrame(data = res, columns = ['class', 'count_P31', 'count_P31/P279*'])
        except Exception as e:
            time.sleep(5)
            print('Try again ', entity_id, f)
            return entities_sparqul(entity_id, f + 1)

In [ ]:
exl = pd.read_excel('.xlsx')
for i in exl['subclass']:
    res.append(entities_sparqul(i))
final_result = pd.concat(res)
final_result.to_excel('.xlsx', encoding='utf-8-sig', index = False)

In [1]:
import dask
from fake_useragent import UserAgent
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import pandas as pd
import json
import time
import bookquery
import os
import io
import json
from tqdm import tqdm
from os.path import join,isfile
from os import listdir
from collections import Counter
import dask


def counter_for_files(path):
    
    """Подсчет файлов в директории"""
        
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

In [11]:
def creating_file(file):
    filename = file.split('\\')[1].split('.')[0]
    if f"{filename}_without_zeros" in list(map(lambda x: x.split('\\')[1].split('.')[0] , counter_for_files(r"excel_with_subclasses/without_zeros"))):
        return
    xl = pd.ExcelFile(file)
    for i in range(len(xl.sheet_names)):
        res = []
        df = xl.parse(xl.sheet_names[i])
        df = df[df["count_P31"]>0]
        if len(df)==0:
            return
        if i==0:
            df.to_excel(f'excel_with_subclasses/without_zeros/{filename}_without_zeros.xlsx', sheet_name = xl.sheet_names[i] ,encoding='utf-8-sig' , index = False)
        else:
            with pd.ExcelWriter(f'excel_with_subclasses/without_zeros/{filename}_without_zeros.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
                df.to_excel(writer, sheet_name = xl.sheet_names[i] , encoding='utf-8-sig' , index = False)
        

In [12]:
for file in tqdm(counter_for_files(r"excel_with_subclasses/with_count")[1:2]):
    creating_file(file)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.76it/s]
